In [81]:
#Importing packages
import requests
import bs4 as bs
import pandas as pd
import re
import datetime
from datetime import datetime

In [181]:
# Getting the URL
url = 'https://en.wikipedia.org/wiki/List_of_post%E2%80%932016_election_Donald_Trump_rallies'
response = requests.get(url, headers={'name': 'Laurits Harbom Marcussen', 'email': 'zhg932@alumni.ku.dk'})
soup = bs.BeautifulSoup(response.content, 'lxml') # Get the HTML of the website
tables = soup.find_all('table')

In [83]:
#Converts html's to dataframes, convert lists to dataframe
df=pd.read_html(str(tables))
df1=pd.DataFrame(df[8])
df2=pd.DataFrame(df[9])

#Isolate the "Date of rally2 columns, and combine the 2 tables
dates = pd.concat([df1["Date of rally"], df2["Date of rally"]])
dates = dates.reset_index(drop=True)

#Cleans wiki-data
def clean():
    for i in range(len(dates)):
        dates[i] = re.sub(r'\[\d+\]', '', dates[i])
        date_list = dates[i].split(',') # # Removing the weekday and the comma from the strings by splitting the strings and merging them again
        dates[i] = ''.join(date_list[1:])
        dates[i] = pd.to_datetime(dates[i]).strftime('%Y-%m-%d') # Converting to datetime format
    return dates
clean()

0     2020-06-20
1     2020-06-23
2     2020-08-17
3     2020-08-17
4     2020-08-18
         ...    
65    2020-11-02
66    2020-11-02
67    2020-12-05
68    2021-01-04
69    2021-01-06
Name: Date of rally, Length: 70, dtype: object

In [107]:
#Craeting a list of all the dates we wanna analyze
all_dates = pd.date_range(start='5/21/2020', end='2/05/2021')
date_list = all_dates.strftime('%Y-%m-%d')
s = pd.Series(date_list)

In [108]:
#Comparing the list of all dates and the list of rally dates, and making a binary list of coinciding dates
binary_list = []
for i in s:
    if i in dates.values: 
        binary_list.append(1)
    else: 
        binary_list.append(0)
binaries = pd.Series(binary_list)

In [109]:
#Making a combined list
rallies = [[s[i], binaries[i]] for i in range(len(binary_list))]

In [110]:
#Making a dataframe with the dates
df_rally = pd.DataFrame(rallies, columns=['date', 'rally'])

In [111]:
#create a date column with datetime
df_rally['date'] = pd.to_datetime(df_rally['date'])

In [112]:
df_rally

,date,rally
0,2020-05-21,0
1,2020-05-22,0
2,2020-05-23,0
3,2020-05-24,0
4,2020-05-25,0
...,...,...
256,2021-02-01,0
257,2021-02-02,0
258,2021-02-03,0
259,2021-02-04,0


In [121]:
last_rally = 79
days_since_list = []

#days_since = last_rally 
for rally in df_rally['rally']:
    if rally == 1:
        last_rally = 0
    else:
        last_rally += 1
    days_since_list.append(last_rally)
#Insert into dataframe
df_rally['days_since_last_rally'] = days_since_list

In [126]:
#Get a reversed dataframe
rally_reverse = df_rally.sort_values(by='date', ascending=False).copy()

In [134]:
days_to_next = []
next_rally = 139
for rally in rally_reverse['rally']:
    if rally == 1:
        next_rally = 0
    else:
        next_rally += 1
    days_to_next.append(next_rally)
days_to_next = list(reversed(days_to_next))
df_rally['days_to_next_rally'] = days_to_next

In [138]:
df_rally.iloc[20:50]

,date,rally,days_since_last_rally,days_to_next_rally
20,2020-06-10,0,100,10
21,2020-06-11,0,101,9
22,2020-06-12,0,102,8
23,2020-06-13,0,103,7
24,2020-06-14,0,104,6
25,2020-06-15,0,105,5
26,2020-06-16,0,106,4
27,2020-06-17,0,107,3
28,2020-06-18,0,108,2
29,2020-06-19,0,109,1


In [176]:
#Get the rallies within three days of the given index
within_3_days = []
cnt = 0
for i in range(len(rallies)):
    for d_r in rallies[(i-3):(i+3)]:
        cnt += d_r[1]
    within_3_days.append(cnt)
    cnt = 0
df_rally['within_3_days'] = within_3_days

In [177]:
df_rally.iloc[25:40]

,date,rally,days_since_last_rally,days_to_next_rally,within_3_days
25,2020-06-15,0,105,5,0
26,2020-06-16,0,106,4,0
27,2020-06-17,0,107,3,0
28,2020-06-18,0,108,2,1
29,2020-06-19,0,109,1,1
30,2020-06-20,1,0,0,1
31,2020-06-21,0,1,2,2
32,2020-06-22,0,2,1,2
33,2020-06-23,1,0,0,2
34,2020-06-24,0,1,54,1


In [180]:
#Load tweet dataframe from csv
df_combined_tweets = pd.read_csv('combined_tweets.csv')
df_combined_tweets.head()
#Convert date to datetime
df_combined_tweets['date'] = pd.to_datetime(df_combined_tweets['date'])
#Merge the two dataframes
merged_df = pd.merge(df_combined_tweets, df_rally, on='date', how='left')
merged_df

,date,tweet_count,total_avg,positive_count,positive_avg,negative_count,negative_avg,day,rally,days_since_last_rally,days_to_next_rally,within_3_days
0,2021-01-08,816,-0.058430,296,0.471341,351,-0.533321,2021-01-08,0,2,168,1
1,2021-01-09,855,-0.055365,305,0.485512,370,-0.528158,2021-01-09,0,3,167,1
2,2021-01-10,598,-0.129875,202,0.463348,297,-0.576639,2021-01-10,0,4,166,0
3,2021-01-11,691,-0.042697,231,0.515897,299,-0.497245,2021-01-11,0,5,165,0
4,2021-01-12,916,-0.055817,318,0.510651,394,-0.541918,2021-01-12,0,6,164,0
5,2021-01-13,712,-0.099969,235,0.478191,337,-0.544667,2021-01-13,0,7,163,0
6,2021-01-14,745,-0.058348,232,0.478220,296,-0.521677,2021-01-14,0,8,162,0
7,2021-01-15,568,-0.054584,183,0.530176,237,-0.540193,2021-01-15,0,9,161,0
8,2021-01-16,359,-0.119093,107,0.455482,168,-0.544589,2021-01-16,0,10,160,0
9,2021-01-17,273,-0.073653,86,0.494123,115,-0.544364,2021-01-17,0,11,159,0
